In [1]:
import numpy as np 
import pandas as pd 
import re 
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [46]:
data=pd.read_csv("train.tsv",sep='\t')

In [47]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [48]:
x=data.Phrase
y=data.Sentiment

In [49]:
x.shape,y.shape

((156060,), (156060,))

In [6]:
import re

In [50]:
sentences=[]
for i in range(len(x)):
    sentence=x.iloc[i].lower()
    sentence=re.sub('[^a-zA-Z]',' ',sentence)
    sentences.append(sentence)

In [51]:
len(sentences)

156060

In [10]:
sentences[0]

'a series of escapades demonstrating the adage that what is good for the goose is also good for the gander   some of which occasionally amuses but none of which amounts to much of a story  '

In [11]:
lemmatizer=WordNetLemmatizer()

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [52]:
corpus=[]
for i in range(len(sentences)):
    words=nltk.word_tokenize(sentences[i])
    words=[lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
    sent=' '.join(words)
    corpus.append(sent)

In [53]:
len(corpus)

156060

In [54]:
corpus[0]

'series escapade demonstrating adage good goose also good gander occasionally amuses none amount much story'

In [23]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

In [55]:
ont_hot_representation=[one_hot(word,10000) for word in corpus]

In [56]:
len(ont_hot_representation)

156060

In [94]:
import tensorflow as tf

In [95]:
@tf.function
def max_len(l):
    max=0
    for each in l:
        if len(each)>max:
            max=len(each)
    return max

In [97]:
max_length_word=max_len(ont_hot_representation)

In [98]:
max_length_word

<tf.Tensor: shape=(), dtype=int32, numpy=30>

In [99]:
padded_sequences=pad_sequences(ont_hot_representation,max_length_word)

In [100]:
len(padded_sequences)

156060

In [101]:
padded_sequences[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 4121, 1871, 7475, 8743, 3760, 7257, 2284,
       3760, 2810, 6690, 7787, 9602, 4970, 1188, 6935], dtype=int32)

In [63]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM


In [102]:
model=Sequential()

In [103]:
model.add(Embedding(10000,256,input_length=max_length_word))
model.add(Dropout(0.2))

In [104]:
model.add(LSTM(256,dropout=0.3,recurrent_dropout=0.3))

In [105]:
model.add(Dense(256,activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.3))

In [106]:
model.add(Dense(5,activation='softmax'))

In [107]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [70]:
from sklearn.model_selection import train_test_split

In [112]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences_final,y_final,test_size=0.2,random_state=0)

In [111]:
padded_sequences_final=np.array(padded_sequences)
y_final=np.array(y_categorical)

In [87]:
from keras.utils import to_categorical

In [109]:
y_categorical= to_categorical(y)

In [110]:
y_categorical

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [116]:
model.fit(X_train,y_train,batch_size=256,epochs=20,validation_split=0.2)

Epoch 1/20
391/391 [==============================] - 60s 153ms/step - loss: 0.7486 - accuracy: 0.6882 - val_loss: 0.8998 - val_accuracy: 0.6410
Epoch 2/20
391/391 [==============================] - 60s 154ms/step - loss: 0.7165 - accuracy: 0.6983 - val_loss: 0.9174 - val_accuracy: 0.6391
Epoch 3/20
391/391 [==============================] - 60s 154ms/step - loss: 0.6924 - accuracy: 0.7071 - val_loss: 0.9292 - val_accuracy: 0.6418
Epoch 4/20
391/391 [==============================] - 61s 157ms/step - loss: 0.6708 - accuracy: 0.7136 - val_loss: 0.9522 - val_accuracy: 0.6379
Epoch 5/20
391/391 [==============================] - 60s 154ms/step - loss: 0.6505 - accuracy: 0.7208 - val_loss: 0.9700 - val_accuracy: 0.6437
Epoch 6/20
391/391 [==============================] - 60s 153ms/step - loss: 0.6314 - accuracy: 0.7258 - val_loss: 1.0100 - val_accuracy: 0.6432
Epoch 7/20
391/391 [==============================] - 59s 152ms/step - loss: 0.6143 - accuracy: 0.7330 - val_loss: 1.0344 - val_ac

In [129]:
y_pred=np.argmax(model.predict(X_test),axis=1)

In [130]:
y_pred[0]

2

In [131]:
model.evaluate(X_test,y_test)

976/976 [==============================] - 10s 10ms/step - loss: 1.5753 - accuracy: 0.6250


[1.5753288269042969, 0.625048041343689]

TypeError: ignored